In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Paraíba - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego,Paraíba - Consumo de Cimento (t)
0,2003,0.656510,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,1475.829837,...,19.809501,80.471176,4.378713,2.441479,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,8.315958,392.755000
1,2004,0.661779,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,1587.871571,...,19.587064,80.859838,4.352905,3.057163,3.061905e+07,1.650726e+06,7.318404,2.750422e+07,8.243195,354.563000
2,2005,0.666544,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,1686.293242,...,19.173043,82.919786,4.533780,3.724982,3.105294e+07,1.663425e+06,7.349948,2.765335e+07,8.219443,365.065000
3,2006,0.670820,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,1796.153586,...,19.147347,83.470926,4.738770,4.383607,3.145782e+07,1.689128e+06,7.378438,2.778800e+07,8.131041,430.470000
4,2007,0.674620,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,1888.085455,...,19.243272,84.316110,4.761987,4.964201,3.179414e+07,1.705364e+06,7.389855,2.785711e+07,8.106365,523.308000
5,2008,0.677959,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,1945.158359,...,19.529327,85.152396,4.537776,5.342951,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,8.076245,707.783000
6,2009,0.680849,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,1991.871625,...,19.708037,85.812083,4.395976,5.534727,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,8.105416,734.672000
7,2010,0.683306,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,2087.894628,...,19.669456,86.266816,4.409553,5.851130,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,8.069181,834.169000
8,2011,0.685344,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,2212.198689,...,19.581458,85.939564,4.376359,6.127922,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,8.055989,946.248000
9,2012,0.686846,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,2280.624032,...,19.382961,84.829158,4.240366,6.316335,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,10.160575,1103.437000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
0,0.656510,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,1475.829837,16.405809,...,23.750637,19.809501,80.471176,4.378713,2.441479,3.017292e+07,1.631334e+06,7.291169,2.736342e+07,8.315958
1,0.661779,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,1587.871571,14.098208,...,23.428125,19.587064,80.859838,4.352905,3.057163,3.061905e+07,1.650726e+06,7.318404,2.750422e+07,8.243195
2,0.666544,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,1686.293242,13.228324,...,22.528887,19.173043,82.919786,4.533780,3.724982,3.105294e+07,1.663425e+06,7.349948,2.765335e+07,8.219443
3,0.670820,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,1796.153586,14.112250,...,22.383216,19.147347,83.470926,4.738770,4.383607,3.145782e+07,1.689128e+06,7.378438,2.778800e+07,8.131041
4,0.674620,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,1888.085455,14.726361,...,22.302562,19.243272,84.316110,4.761987,4.964201,3.179414e+07,1.705364e+06,7.389855,2.785711e+07,8.106365
5,0.677959,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,1945.158359,15.160191,...,22.419324,19.529327,85.152396,4.537776,5.342951,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,8.076245
6,0.680849,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,1991.871625,15.411234,...,22.467973,19.708037,85.812083,4.395976,5.534727,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,8.105416
7,0.683306,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,2087.894628,15.193321,...,22.336913,19.669456,86.266816,4.409553,5.851130,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,8.069181
8,0.685344,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,2212.198689,14.487239,...,22.322079,19.581458,85.939564,4.376359,6.127922,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,8.055989
9,0.686846,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,2280.624032,13.985942,...,22.350186,19.382961,84.829158,4.240366,6.316335,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,10.160575


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      354.563000
1      365.065000
2      430.470000
3      523.308000
4      707.783000
5      734.672000
6      834.169000
7      946.248000
8     1103.437000
9     1223.444000
10    1240.232857
11    1076.248286
12     928.236857
13     786.338000
14     707.565000
15     769.080000
16    1051.694000
17    1112.087000
18    1064.438000
19            NaN
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
0,-2.021610,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.763177,2.532042,...,3.028943,1.822703,-1.753819,0.335293,-2.288696,-1.647449,-2.273161,-2.121047,-2.002019,-0.806111
1,-1.618397,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.454969,-0.330364,...,2.196651,1.010731,-1.524773,0.242365,-1.839656,-1.433705,-1.734803,-1.707018,-1.626108,-0.858247
2,-1.253728,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.184227,-1.409389,...,-0.123976,-0.500587,-0.310809,0.893640,-1.352592,-1.225822,-1.382234,-1.227486,-1.227951,-0.875266
3,-0.926533,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.882019,-0.312945,...,-0.499903,-0.594386,0.013988,1.631745,-0.872234,-1.031838,-0.668651,-0.794371,-0.868441,-0.938608
4,-0.635739,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.629130,0.448813,...,-0.708045,-0.244228,0.512070,1.715343,-0.448787,-0.870702,-0.217902,-0.620818,-0.683925,-0.956288
5,-0.380275,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.472131,0.986947,...,-0.406720,0.799970,1.004909,0.908028,-0.172551,-0.729197,0.289934,-0.466668,-0.555978,-0.977870
6,-0.159069,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.343631,1.298346,...,-0.281173,1.452322,1.393674,0.397450,-0.032682,-0.629030,0.679792,-0.509224,-0.569210,-0.956968
7,0.028948,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.079487,1.028042,...,-0.619395,1.311490,1.661656,0.446338,0.198081,-0.250333,1.072005,-0.041110,-0.104766,-0.982932
8,0.184850,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.262453,0.152201,...,-0.657676,0.990266,1.468801,0.326814,0.399956,0.109738,1.182094,0.422316,0.328924,-0.992384
9,0.299823,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.450680,-0.469621,...,-0.585143,0.265683,0.814419,-0.162854,0.537372,0.441076,1.280980,0.783687,0.710858,0.515590


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      354.563000
1      365.065000
2      430.470000
3      523.308000
4      707.783000
5      734.672000
6      834.169000
7      946.248000
8     1103.437000
9     1223.444000
10    1240.232857
11    1076.248286
12     928.236857
13     786.338000
14     707.565000
15     769.080000
16    1051.694000
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
17,0.98374,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,2.45631,0.938425,...,0.228925,-0.446315,-0.563593,-2.618397,1.406933,0.77241,-0.009612,0.301112,0.401403,1.165086


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1112.087
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [13]:
train_input.iloc[-(len(train_input)//5):]

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
14,1.345395,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,0.651544,-0.608012,...,-0.048865,-1.228016,-0.918096,-0.795425,0.836267,1.188614,-0.094510,0.908155,0.969461,1.175737
15,1.230676,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.322697,0.245785,...,0.154656,-0.748168,-0.718077,-1.328898,0.851838,1.134725,-0.144853,0.795288,0.824407,1.197052
16,1.147268,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,2.374644,0.196199,...,0.214200,-0.583918,-0.648297,-2.317593,1.141098,1.020077,-0.057621,0.603047,0.668766,1.254680


In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2792038856, 2636397904, 2281762519, 894193299, 2951945524, 2064954622, 1698248851, 3488752655, 775879868, 2359432387, 2835430242, 2451366092, 1893959707, 2237643115, 3904930601, 1179819442, 2388286567, 2907526740, 1918562619, 871483675, 236809486, 1577889984, 3033846116, 3586203848, 1028453453, 3583104869, 4155883669, 3354886017, 1885170364, 305127302, 315263953, 4183089899, 4152471543, 3001510387, 936893998, 3599111141, 2108309360, 3749026554, 1477552093, 2086017773, 2929799423, 61864791, 4235577600, 4100886923, 1994125416, 3404423462, 3301057870, 1046185111, 2580278108, 1353258745, 3952084059, 3916194481, 1135926481, 1282357434, 1288604403, 3489235481, 1961094310, 3844197899, 3081275789, 1227378099, 1531723530, 2224116737, 713985783, 4288248561, 1649603603, 3626602630, 3667241102, 2274252392, 3584883874, 2587643644, 2600963691, 1132833428, 1291977597, 2468815669, 3426069148, 1398713934, 3516741405, 1754080823, 95296095, 4135088884, 3329031925, 3417850133, 779895118, 1946472211, 1377

val_loss: 23403.384765625


Step: 86 ___________________________________________
val_loss: 22597.240234375


Step: 87 ___________________________________________
val_loss: 24223.8046875


Step: 88 ___________________________________________
val_loss: 22453.34375


Step: 89 ___________________________________________
val_loss: 22453.349609375


Step: 90 ___________________________________________
val_loss: 22539.427734375


Step: 91 ___________________________________________
val_loss: 22453.275390625


Step: 92 ___________________________________________
val_loss: 22453.279296875


Step: 93 ___________________________________________
val_loss: 26756.6171875


Step: 94 ___________________________________________
val_loss: 22863.115234375


Step: 95 ___________________________________________
val_loss: 22463.556640625


Step: 96 ___________________________________________
val_loss: 22569.138671875


Step: 97 ___________________________________________
val_loss: 22496.166015625


Step: 98

In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 447ms/step - loss: 731650.5625 - val_loss: 693848.6875
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 701738.5000 - val_loss: 334609.9062
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 654130.8750 - val_loss: 1009309.7500
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 697420.7500 - val_loss: 190662.7500
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 460395.0000 - val_loss: 914310.1875
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 426477.2188 - val_loss: 891200.5000
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 510715.3438 - val_loss: 1098615.1250
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 522339.2812 - val_loss: 715790.7500
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 453819.6562 - val_loss: 428897.1562
Epoch 1

1/1 [==============================] - 0s 13ms/step - loss: 142622.0938 - val_loss: 117404.2578
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 140525.1562 - val_loss: 103283.5547
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 126215.2500 - val_loss: 131681.5312
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 130083.4531 - val_loss: 82709.3750
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 125425.9375 - val_loss: 84897.0547
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 123680.6016 - val_loss: 89944.1172
Epoch 81/10000
1/1 [==============================] - 0s 17ms/step - loss: 117380.8828 - val_loss: 96547.8828
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 121240.3906 - val_loss: 94387.9375
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 118239.9844 - val_loss: 96119.3828
Epoch 84/10000
1/1 [==

1/1 [==============================] - 0s 13ms/step - loss: 60799.0781 - val_loss: 27730.5488
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 55217.5586 - val_loss: 30587.8066
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 57248.7969 - val_loss: 27568.3926
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 49909.3477 - val_loss: 28034.9844
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 46337.1133 - val_loss: 28542.0625
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 51762.7148 - val_loss: 28650.8262
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 49863.4453 - val_loss: 28234.5527
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 50466.1680 - val_loss: 28895.3809
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 50961.8633 - val_loss: 28075.6230
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 34013.2539 - val_loss: 30336.3418
Epoch 224/10000
1/1 [==============================] - 0s 16ms/step - loss: 34094.8320 - val_loss: 33622.3086
Epoch 225/10000
1/1 [==============================] - 0s 15ms/step - loss: 30733.8672 - val_loss: 35677.8672
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 30625.1074 - val_loss: 33503.8750
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 26747.7676 - val_loss: 32744.2051
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 27730.5781 - val_loss: 28013.3340
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 24661.2227 - val_loss: 28003.8535
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 26010.7129 - val_loss: 37294.0664
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 24541.7617 - val_loss: 33917.0781
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 14494.0293 - val_loss: 24403.2832
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 13425.6104 - val_loss: 23038.2266
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 12463.7920 - val_loss: 22829.8594
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 14391.4082 - val_loss: 22801.6270
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 14978.8506 - val_loss: 23027.5000
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 11139.1953 - val_loss: 23188.0723
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 12200.6523 - val_loss: 23618.5488
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 15078.6465 - val_loss: 23091.7910
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 16512.9805 - val_loss: 23991.7891
Epoch 306/10000
1/1 [=====

...

Epoch 746/10000
1/1 [==============================] - 0s 13ms/step - loss: 12711.5020 - val_loss: 23018.3770
Epoch 747/10000
1/1 [==============================] - 0s 13ms/step - loss: 12981.2773 - val_loss: 23330.8848
Epoch 748/10000
1/1 [==============================] - 0s 13ms/step - loss: 12331.4873 - val_loss: 22617.4844
Epoch 749/10000
1/1 [==============================] - 0s 13ms/step - loss: 12895.4912 - val_loss: 23230.2559
Epoch 750/10000
1/1 [==============================] - 0s 13ms/step - loss: 12615.6406 - val_loss: 22822.8652
Epoch 751/10000
1/1 [==============================] - 0s 13ms/step - loss: 10480.8076 - val_loss: 22703.1074
Epoch 752/10000
1/1 [==============================] - 0s 14ms/step - loss: 11585.2910 - val_loss: 23768.7285
Epoch 753/10000
1/1 [==============================] - 0s 13ms/step - loss: 12714.6934 - val_loss: 23043.8125
Epoch 754/10000
1/1 [==============================] - 0s 13ms/step - loss: 12938.9072 - val_loss: 23282.4121
Epoch 755/

1/1 [==============================] - 0s 13ms/step - loss: 9644.0293 - val_loss: 24374.9629
Epoch 821/10000
1/1 [==============================] - 0s 13ms/step - loss: 9355.4756 - val_loss: 24408.0078
Epoch 822/10000
1/1 [==============================] - 0s 13ms/step - loss: 10077.4561 - val_loss: 24295.2754
Epoch 823/10000
1/1 [==============================] - 0s 13ms/step - loss: 11984.5752 - val_loss: 25824.7578
Epoch 824/10000
1/1 [==============================] - 0s 13ms/step - loss: 9219.7480 - val_loss: 24644.0938
Epoch 825/10000
1/1 [==============================] - 0s 13ms/step - loss: 11753.2393 - val_loss: 25402.7051
Epoch 826/10000
1/1 [==============================] - 0s 14ms/step - loss: 9118.9502 - val_loss: 24409.9297
Epoch 827/10000
1/1 [==============================] - 0s 14ms/step - loss: 12299.8564 - val_loss: 25269.7090
Epoch 828/10000
1/1 [==============================] - 0s 13ms/step - loss: 9665.6699 - val_loss: 24147.0723
Epoch 829/10000
1/1 [==========

Epoch 895/10000
1/1 [==============================] - 0s 13ms/step - loss: 15949.9648 - val_loss: 35422.9766
Epoch 896/10000
1/1 [==============================] - 0s 13ms/step - loss: 18429.9746 - val_loss: 31283.1621
Epoch 897/10000
1/1 [==============================] - 0s 13ms/step - loss: 17997.1855 - val_loss: 32229.1309
Epoch 898/10000
1/1 [==============================] - 0s 13ms/step - loss: 17685.9043 - val_loss: 32387.6016
Epoch 899/10000
1/1 [==============================] - 0s 13ms/step - loss: 17652.6484 - val_loss: 32595.1582
Epoch 900/10000
1/1 [==============================] - 0s 13ms/step - loss: 17613.3320 - val_loss: 32714.3828
Epoch 901/10000
1/1 [==============================] - 0s 13ms/step - loss: 17791.3496 - val_loss: 32799.7227
Epoch 902/10000
1/1 [==============================] - 0s 13ms/step - loss: 17872.8730 - val_loss: 32410.8047
Epoch 903/10000
1/1 [==============================] - 0s 13ms/step - loss: 17849.6426 - val_loss: 32423.8535
Epoch 904/

1/1 [==============================] - 0s 13ms/step - loss: 13425.4990 - val_loss: 26550.5410
Epoch 970/10000
1/1 [==============================] - 0s 13ms/step - loss: 15015.5352 - val_loss: 26527.9375
Epoch 971/10000
1/1 [==============================] - 0s 13ms/step - loss: 13436.5537 - val_loss: 26385.1562
Epoch 972/10000
1/1 [==============================] - 0s 14ms/step - loss: 13552.5977 - val_loss: 26630.0371
Epoch 973/10000
1/1 [==============================] - 0s 14ms/step - loss: 13358.3389 - val_loss: 26330.4141
Epoch 974/10000
1/1 [==============================] - 0s 13ms/step - loss: 13368.5527 - val_loss: 26374.7324
Epoch 975/10000
1/1 [==============================] - 0s 13ms/step - loss: 14535.6348 - val_loss: 26036.5527
Epoch 976/10000
1/1 [==============================] - 0s 13ms/step - loss: 14519.1943 - val_loss: 26058.6113
Epoch 977/10000
1/1 [==============================] - 0s 13ms/step - loss: 14474.2109 - val_loss: 26010.8691
Epoch 978/10000
1/1 [=====

1/1 [==============================] - 0s 23ms/step - loss: 12567.0234 - val_loss: 24783.1250
Epoch 1043: early stopping


In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[1112.087](test_target) - [[868.623]](prediction) = 243.46401416015624


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
0,-1.704907,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.681319,1.766479,...,2.005673,1.343770,-1.577117,-0.894671,-1.704754,-1.577007,-1.556934,-1.816118,-1.791670,1.871815
1,-1.107554,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.113171,-0.759367,...,1.388701,0.427224,-1.384812,-1.065747,-1.172252,-1.074050,-1.076262,-1.181625,-1.148529,1.007625
2,-0.567302,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.614089,-1.711522,...,-0.331560,-1.278737,-0.365576,0.133221,-0.594658,-0.584887,-0.761471,-0.446748,-0.467330,0.725527
3,-0.082567,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.057003,-0.743997,...,-0.610232,-1.384617,-0.092879,1.492037,-0.025015,-0.128428,-0.124350,0.216995,0.147749,-0.324413
4,0.348240,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.409171,-0.071806,...,-0.764526,-0.989362,0.325307,1.645937,0.477138,0.250739,0.278100,0.482963,0.463435,-0.617481
5,0.726707,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.698580,0.403055,...,-0.541156,0.189318,0.739090,0.159707,0.804717,0.583710,0.731521,0.719195,0.682338,-0.975215
6,1.054419,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.935456,0.677840,...,-0.448089,0.925688,1.065495,-0.780242,0.970584,0.819410,1.079605,0.653979,0.659698,-0.628747
7,1.332964,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.422375,0.439318,...,-0.698810,0.766717,1.290491,-0.690243,1.244240,1.710514,1.429792,1.371358,1.454309,-1.059112


train_target:


,Paraíba - Consumo de Cimento (t)
0,354.563
1,365.065
2,430.470
3,523.308
4,707.783
5,734.672
6,834.169
7,946.248


test_input:


...

,Paraíba - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraíba - Produção de Cimento (t),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Valor Cimento Portland,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Desemprego
17,0.98374,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,2.45631,0.938425,...,0.228925,-0.446315,-0.563593,-2.618397,1.406933,0.77241,-0.009612,0.301112,0.401403,1.165086


test_target:


,Paraíba - Consumo de Cimento (t)
17,1112.087


1/1 [==============================] - 0s 37ms/step
Error: 243.46401416015624




[700.3123168945312,
 952.5711669921875,
 867.5780639648438,
 1027.95361328125,
 1164.005859375,
 1041.90185546875,
 835.7778930664062,
 916.136962890625,
 772.2677001953125,
 868.6229858398438]

In [22]:
display(targets)
display(predictions)

[1103.437,
 1223.444,
 1240.2328571428573,
 1076.248285714286,
 928.2368571428572,
 786.338,
 707.565,
 769.08,
 1051.694,
 1112.087]

[700.3123168945312,
 952.5711669921875,
 867.5780639648438,
 1027.95361328125,
 1164.005859375,
 1041.90185546875,
 835.7778930664062,
 916.136962890625,
 772.2677001953125,
 868.6229858398438]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

238.44400093470986

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.23848304060845749